In [4]:
# Transfer learning with elmo embedding
# Import our dependencies


from __future__ import division

import numpy as np
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Input, Dropout
from keras.models import Model, load_model
import keras.layers as layers
from keras.engine import Layer
from keras.optimizers import Adam
from keras.regularizers import l2
import os
import re
from keras import backend as K

from graph_attention_layer import GraphAttention

import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub


import numpy as np

# Initialize session
sess = tf.Session()
K.set_session(sess)

In [21]:
# # Read data
# A, X, Y_train, Y_val, Y_test, idx_train, idx_val, idx_test = load_data('cora')

X = np.random.uniform(0,9,[10,32])
A = np.random.uniform(1,-1,[10,10])

# # Parameters
N = X.shape[0]                # Number of nodes in the graph
F = X.shape[1]                # Original feature dimension
n_classes = 10  # Number of classes
F_ = 8                        # Output size of first GraphAttention layer
n_attn_heads = 8              # Number of attention heads in first GAT layer
dropout_rate = 0.6            # Dropout rate (between and inside GAT layers)
l2_reg = 5e-4/2               # Factor for l2 regularization
# learning_rate = 5e-3          # Learning rate for Adam
# epochs = 10000                # Number of training epochs
# es_patience = 100             # Patience fot early stopping

# # Preprocessing operations
# X = preprocess_features(X)
A = A + np.eye(A.shape[0])  # Add self-loops

# # Model definition (as per Section 3.3 of the paper)

# constants = [1,2,3]
# k_constants = K.variable(constants)
# fixed_input = Input(tensor=k_constants)

k_constants = K.variable(X)
X_in = layers.Input(tensor=k_constants)
ks_constants = K.variable(A)
A_in = layers.Input(tensor=ks_constants)



dropout1 = Dropout(dropout_rate)(X_in)
graph_attention_1 = GraphAttention(F_,
                                   attn_heads=n_attn_heads,
                                   attn_heads_reduction='concat',
                                   dropout_rate=dropout_rate,
                                   activation='elu',
                                   kernel_regularizer=l2(l2_reg),
                                   attn_kernel_regularizer=l2(l2_reg))([dropout1, A_in])
dropout2 = Dropout(dropout_rate)(graph_attention_1)
graph_attention_2 = GraphAttention(50,
                                   attn_heads=1,
                                   attn_heads_reduction='average',
                                   dropout_rate=dropout_rate,
                                   activation='softmax',
                                   kernel_regularizer=l2(l2_reg),
                                   attn_kernel_regularizer=l2(l2_reg))([dropout2, A_in])

new_model = Model([X_in, A_in], graph_attention_2, name='new_model')



In [12]:
new_model.output

<tf.Tensor 'graph_attention_6/Softmax_1:0' shape=(10, 50) dtype=float32>

In [22]:
from keras.layers import Embedding, LSTM, Dense

#general vars
length = 150


dic_size = 100
embed_size = 12

input_text = Input(shape=(length,))
embedding = Embedding(dic_size, embed_size)(input_text)

embedding = LSTM(5)(embedding) 
embedding = Dense(50)(embedding)

model_a = Model(input_text, embedding, name = 'model_a')

embedding

<tf.Tensor 'dense_5/BiasAdd:0' shape=(?, 50) dtype=float32>

In [33]:
mult = Lambda(lambda x: tf.matmul(x[0], x[1], transpose_b=True))([embedding, graph_attention_2])

# model = Model([input_text, dummyInput], mult, name='full_model')

final_model = Model(inputs=[model_a.input,new_model.input[0],new_model.input[1]], outputs=mult)

final_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
final_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (10, 32)             0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 150)          0                                            
__________________________________________________________________________________________________
dropout_81 (Dropout)            (10, 32)             0           input_16[0][0]                   
__________________________________________________________________________________________________
input_17 (InputLayer)           (10, 10)             0                                            
__________________________________________________________________________________________________
embedding_

In [ ]:
# Function to build model

num_classes = len(set(final['labels']))

def build_model(): 
    input_text = layers.Input(shape=(1,), dtype="string")
    embedding = ElmoEmbeddingLayer()(input_text)
    dense = layers.Dense(256, activation='relu')(embedding)
    pred = layers.Dense(num_classes, activation='softmax')(dense)
    model = Model(inputs=[input_text], outputs=pred)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model